# Canonical Implementation of Logical Controlled Phase Operators

This notebook illustrates how to implement arbitrary logical CP operators on any CSS code using the embedded code technique.
    
**Input:** 

- The X-checks $S_X$ and X-logicals $L_X$ of a CSS code;
- The desired logical action $B$ expressed as a CP operator on $k$ qubits.
    
**Output:** 

- Canonical implementation of $B$ on the code using multi-qubit CP and RP gates (not in general depth-one)
- An embedded code which has an implementation of $B$ using single-qubit gates

Users can run the algorithm on a variety of codes, including:
- 2D Toric code
- Hypercube codes
- Reed Muller codes
- Codes from http://codetables.de/TableIII.php - cut and paste into the code below
- LDPC codes


In [1]:
from add_parent_dir import *
from common import *
from NHow import *
from CSSLO import *
import itertools as iter

########################################################
## default values
########################################################
SX,LX,SZ,LZ = None,None,None,None
t= 1
target = None


########################################################
## 2D toric Code
########################################################
SX, SZ = toric2D(3)
# target = 'S[0]'
# target = 'T[0]'
# target = 'CZ[0,1]'
target = 'CS[0,1]'
# target = 'S[0]S3[1]'

########################################################
## Codetables code 
########################################################

## paste into mystr from codetables.de website
## examples

## Steane Code
# mystr = '''      [1 0 0 1 0 1 1|0 0 0 0 0 0 0]
#       [0 1 0 1 1 1 0|0 0 0 0 0 0 0]
#       [0 0 1 0 1 1 1|0 0 0 0 0 0 0]
#       [0 0 0 0 0 0 0|1 0 0 1 0 1 1]
#       [0 0 0 0 0 0 0|0 1 0 1 1 1 0]
#       [0 0 0 0 0 0 0|0 0 1 0 1 1 1]'''

# SX,SZ,SXZ = CodeTable(mystr)

########################################################
## Bring's code from Fold-Transversal Gates for Quantum Codes
########################################################
# SXx = [[17,24,9,7,29],
# [9,1,15,3,5],
# [7,1,12,2,4],
# [15,12,23,25,28],
# [2,16,10,6,23],
# [4,29,11,18,16],
# [3,28,8,13,19],
# [25,6,26,21,8],
# [10,18,22,30,26],
# [24,5,19,20,14],
# [20,13,21,30,27],
# [17,14,27,22,11]]
# LXx = [[1,6,11],[2,8,22],[4,5,25],[10,13,17],[24,28,30],[12,19,26],[9,16,27],[15,20,29]]
# n = 31
# ## convert to binary vectors, eliminate first col (above start from index 1)
# SX = ZMat([set2Bin(n,x) for x in SXx])[:,1:]
# LX = ZMat([set2Bin(n,x) for x in LXx])[:,1:]
# target = 'S[0]'
# target = 'CZ[0,1]'
# target = 'S[0]S3[1]'


###########################################################
## Hyperbolic Tesselations
###########################################################

## Hyperbolic Quantum Colour Codes https://doi.org/10.48550/arXiv.1804.06382

## 2D
# myfile = "RG-3-6.txt"
# myfile = "RG-3-8.txt"
# myfile = "RG-3-10.txt"
# myfile = "RG-3-12.txt"
# myfile = "RG-3-14.txt"

## 3D
# myfile = 'FG-3-3-3-3.txt'
# myfile = 'FG-3-4-3-4.txt'

# ## Other Hyperbolic Tesselations
# ## Hyperbolic and Semi-Hyperbolic Surface Codes for Quantum Storage  https://doi.org/10.1088/2058-9565/aa7d3b
# myfile = "RG-4-5.txt"
# myfile = "RG-5-5.txt"
# myfile = "RG-7-7.txt"

# codeList = importRGList(myfile)
# print(printRGList(codeList,myfile))
# ## pick the code by varying ix
# ix = 6
# myrow = codeList[ix]

# ## Colour Code
# SX, SZ = complex2ColourCode(myrow[1])

# target = 'CZ[0,1]'

#########################################################################
## Poset Codes from Self-Orthogonal Codes Constructed from Posets and Their Applications in Quantum Communication
## https://doi.org/10.3390/math8091495
#########################################################################

## Form 1: Quantum Hamming codes
## a1 >= 3; a2 = b1 = b2 = 0
# a1,a2,b1,b2 = 4,0,0,0

## Form 2: a1 > a2 >= 3; b1 = b2 = 0
# a1,a2,b1,b2 = 5,3,0,0

## Form 3: a1 = 1; b1 >= 3; a2 = b2 =0
# a1,a2,b1,b2 = 1,0,4,0

# ## Form 4: a1 = a2 = 1; b1 > b2 >= 3
# a1,a2,b1,b2 = 1,1,5,3

# SX = posetCode(a1,a2,b1,b2)
# SZ = SX

# target = 'CZ[0,1]'

####################################################################
## Symmetric Hypergraph Product Codes
## Partitioning qubits in hypergraph product codes to implement logical gates https://doi.org/10.48550/arXiv.2204.10812
####################################################################

## Pattern in the SHPC paper
# T = genH(3)

# ## [[98,32,3]]
# T = '''1110100
# 1011010
# 0111001'''

# ## [[98,18,4]]
# T = '''1101000
# 1010100
# 0110010
# 1110001'''

# SX, SZ = SHPC(T)

# target = 'CZ[0,1]'

####################################################################
####################################################################

SX,LX,SZ,LZ = CSSCode(SX,LX,SZ,LZ)

r,n = np.shape(SX)
k,n = np.shape(LX)

compact = n > 15
print(f'Target Logical Operator: {target}')
print('\nINPUT CSS CODE')
print('Checks and Logicals')
print_SXLX(SX,LX,SZ,LZ,compact)

(x,q1), V1, t = Str2CP(target,n=k)
N = 1 << t

## Setting Vto=None gives the smallest possible V1, V2. 
Vto = None
print(f'\nCANONICAL IMPLEMENTATON OF {target}')
(q2,V2),(q1,V1) = canonical_logical_algorithm(q1,V1,LZ,t,Vto)

## Find auxillia qubits represented by u in U with wt(u) > 1
## Comment out to find underlying code
ix = np.sum(V1,axis=-1) > 1
V1 = np.vstack([ZMatI(n),V1[ix]])

SX_V = RemoveZeroRows(matMul(SX, V1.T, 2)) 
LX_V = RemoveZeroRows(matMul(LX, V1.T, 2))

# R, UList = HowResU(V1.T,np.vstack([SX_V,LX_V]),2)

print('\nEMBEDDED CODE')
print('Qubits in Embedded Code:',len(V1))
print('Number of auxillia qubits required:', len(V1) - n)

SX,LX,SZ,LZ = CSSCode(SX_V,LX_V)


print('Embedded Code Checks and Logicals')
print_SXLX(SX,LX,SZ,LZ,compact)

## Logical identities
K_M = getKM(SX, LX, N)

## Algorithm 1 - search
print(f'\nChecking embedded code for transversal {target} via Kernel Search Method:')
z = ker_search(target,LX,SX,t,debug=True)


Target Logical Operator: CS[0,1]

INPUT CSS CODE
Checks and Logicals
SX
 X[0][3][9][10]
 X[1][4][9][11]
 X[2][5][10][11]
 X[0][6][12][13]
 X[1][7][12][14]
 X[2][8][13][14]
 X[3][6][15][16]
 X[4][7][15][17]
 X[5][8][16][17]
SZ
 X[0][1][9][12]
 X[0][2][10][13]
 X[1][2][11][14]
 X[3][4][9][15]
 X[3][5][10][16]
 X[4][5][11][17]
 X[6][7][12][15]
 X[6][8][13][16]
 X[7][8][14][17]
LX
 X[6][7][8]
 X[11][14][17]
LZ
 X[2][5][8]
 X[15][16][17]

CANONICAL IMPLEMENTATON OF CS[0,1]
Canonical Logical Operator Implementation - CP
 CS[2,15][2,16][2,17][5,15][5,16][5,17][8,15][8,16][8,17] CCZ[2,5,15][2,5,16][2,5,17][2,8,15][2,8,16][2,8,17][2,15,16][2,15,17][2,16,17][5,8,15][5,8,16][5,8,17][5,15,16][5,15,17][5,16,17][8,15,16][8,15,17][8,16,17]
Canonical Logical Operator Implementation - RP
 Z[2][5][8][15][16][17] RT5[2,5][2,8][5,8][15,16][15,17][16,17] RT3[2,15][2,16][2,17][5,15][5,16][5,17][8,15][8,16][8,17] RRT[2,5,15][2,5,16][2,5,17][2,8,15][2,8,16][2,8,17][2,15,16][2,15,17][2,16,17][5,8,15][5,8,16][5